<span style="display: inline-block; margin-right: 10px; vertical-align: middle; border: 1px solid #F8F8F8; border-radius: 8px; padding: 5px; background-color: #F8F8F8;">
    <a href="https://www.designsafe-ci.org/workspace/jupyter-lab-hpc-cuda-ds?appVersion=1.1.1" target="_parent" style="text-decoration: none;">
        <span style="font-family: Helvetica, sans-serif; font-size: 13px; color: #565656; margin-right: 5px; vertical-align: middle; font-weight: 600;">Open in</span>
        <img src="https://www.designsafe-ci.org/media/filer_public/2d/d3/2dd37fbf-289e-49cf-9c1a-879c864c4e17/nsf_nheri-ds.png" alt="Open in DesignSafe" style="width: 100px; height: auto; vertical-align: middle;">
    </a>
</span>

<span style="display: inline-block; margin-right: 10px; vertical-align: middle;">
    <a href="https://colab.research.google.com/github/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/chimney_detector.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="width: 130px; height: auto; vertical-align: middle;"/>
    </a>
</span>

<span style="display: inline-block; vertical-align: middle; margin-top: 6px;">
    <a target="_blank" href="https://lightning.ai/new?repo_url=https%3A//github.com/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/chimney_detector.ipynb">
        <img src="https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/app-2/studio-badge.svg" alt="Open in Studio" style="width: 120px; height: auto; vertical-align: middle;"/>
    </a>
</span>

## **Install BRAILS++**
BRAILS++ is not yet available on PyPi. The following line installs the latest version from the GitHub repository using `pip`.

In [1]:
!pip install git+https://github.com/bacetiner/BrailsPlusPlus

  Cloning https://github.com/bacetiner/BrailsPlusPlus to /tmp/pip-req-build-r_d0cxz0
  Running command git clone --filter=blob:none --quiet https://github.com/bacetiner/BrailsPlusPlus /tmp/pip-req-build-r_d0cxz0
  Resolved https://github.com/bacetiner/BrailsPlusPlus to commit 7d44119cb784cf4dfc32af5a22eb58517d229411
  Preparing metadata (setup.py) ... done


## **Import ImageSet and Importer Methods of BRAILS++**
BRAILS++ offers various Artificial Intelligence-powered modules for predicting building and infrastructure attributes from images, such as the `ChimneyDetector` demonstrated in this example.

The `Importer` class provides the main methods to load these modules. To gather the images needed for attribute prediction, users can use the ImageSet class or BRAILS++ scrapers, which facilitate automated asset geometry and image retrieval. This example illustrates both methods for sourcing imagery for use with the `ChimneyDetector` module.

In [2]:
from brails import Importer
from brails.types.image_set import ImageSet

## Define Google API Key
You need a Google API Key with Street View Static API enabled to download the Google Street View imagery required for this example. Please follow this [link](https://support.google.com/googleapi/answer/6158862?hl=en) for instructions on setting up a Google API key.

In [3]:
API_KEY = 'YOUR-API-KEY-HERE'

## **Load Street-Level Imagery: (Option 1) Download Imagery by Specifying the Region Name**
The `ChimneyDetector` class is created to identify the existence of chimneys in buildings from street-level imagery. In this option, BRAILS++ retrieves the required imagery by following the steps below.
1. <strong>Call the `RegionBoundary` class with the necessary details to specify the region of interest.</strong> In this example, street-level imagery of buildings in Atlantic County, NJ, are downloaded. Therefore, the required information includes the `type` of information provided and the `data` itself, which are the `locationName` and `Atlantic County, NJ`.

2. <strong>Download the footprint inventory for the `RegionBoundary` saved in `region_boundary_object` by downloading the OpenStreetMap footprint data through the `OSM_FootprintScraper`.</strong> The only information required to initialize `OSM_FootprintScraper` is the desired output unit for `length`, which, in this case, is `ft`. The output of `OSM_FootprintScraper` is an `AssetInventory` saved in `atlantic_fp`.

3. <strong>Get a randomly selected `100` building subset of the footprint inventory obtained using `OSM_FootprintScraper`. </strong> This subset is obtained using the `get_random_sample` method of an `AssetInventory` object and is stored in `atlantic_fp_subset`. For this example, the random seed is set to `50` for reproducibility of the results.

4. <strong>Get the street-level imagery for the selected subset using `get_images` method of `GoogleStreet` module.</strong> `get_images` requires two inputs 1) AssetInventory for which the images will be retrieved (in this example, `atlantic_fp_subset`) and 2) the path to the folder where the retrieved images will be saved, which in this case is `tmp/street/`.


In [4]:
# Select a region and create its RegionBoundary:
importer = Importer()

region_data = {"type": "locationName",
               "data": "Atlantic County, NJ"}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

osm_fp_class = importer.get_class("OSM_FootprintScraper")
osm_fp_data = {"length": "ft"}
footprint_scraper = osm_fp_class(osm_fp_data)
atlantic_fp = footprint_scraper.get_footprints(region_boundary_object)

# Subsample from the extracted assets to keep the image downloading step quick.
# Here, we are randomly sampling 100 buildings using a random seed value of 50:
atlantic_fp_subset = atlantic_fp.get_random_sample(100, 50)

# Get street-level imagery for the selected subset using GoogleStreetview:
google_street_class = importer.get_class("GoogleStreetview")
google_street = google_street_class({'apiKey': API_KEY})
images_street = google_street.get_images(
    atlantic_fp_subset, 'tmp/street/')


Searching for Atlantic County, NJ...
Found Atlantic County, New Jersey, United States

Found a total of 18900 building footprints in Atlantic County


INFO:brails.scrapers.google_streetview.google_streetview:
Images will be saved to: /home/bacetiner/Documents/BrailsPlusPlus/examples/image_processor/tmp/street

Obtaining street-level imagery:   1%|█▏                                                                                                                       | 1/100 [00:00<00:11,  8.33it/s]INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39.3609, -74.5711
INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39.4948, -74.5723
INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39.4187, -74.6182
INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39.4750, -74.5095
INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39

## **Predict Existence of Chimneys in A Building Using the ChimneyDetector Module**
`ChimneyDetector` includes a pre-trained EfficientDet_D4 model that has been trained on a custom dataset of 40,000 images. This model is capable of identifying the existence of chimneys.

In [5]:
importer = Importer()
chimney_detector_class = importer.get_class('ChimneyDetector')
chimney_detector = chimney_detector_class()
predictions = chimney_detector.predict(images_street)


Checking the existence of chimneys for each building...
Default chimney detector model in tmp/models/efficientdet-d4_chimneyDetector.pth loaded

Performing chimney detections...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  8.11it/s]


Total execution time: 00:00:12.73


## **Show Prediction Results**

In [6]:
print(predictions)

{16303: 0, 8721: 0, 11930: 0, 7945: 0, 15504: None, 10815: 0, 2791: 0, 17645: 0, 10399: 0, 7352: 0, 18241: 0, 2793: 0, 5023: 0, 11376: None, 3227: 0, 11387: None, 10464: 0, 7287: None, 6192: 0, 2252: 0, 10778: 0, 14053: None, 2914: 0, 2097: 0, 10953: None, 14099: None, 187: 0, 7445: 0, 11057: None, 8722: 0, 14234: None, 3466: 0, 6588: None, 14506: 0, 8882: None, 15073: None, 4149: 0, 8836: 0, 6856: 0, 17501: 0, 4024: 0, 13492: 0, 3851: 0, 17819: None, 16256: 0, 13989: 0, 16958: 0, 17426: 0, 4099: 0, 16364: 0, 18692: 0, 16927: 0, 9847: 0, 3052: None, 12050: None, 16932: 0, 4922: 0, 220: 0, 11743: None, 295: 0, 13466: 0, 15490: 0, 17646: 0, 7346: 0, 5317: 0, 18773: 0, 3208: 0, 4484: 0, 8393: 0, 12323: 0, 8052: 0, 11008: 0, 1097: 0, 16969: 0, 1545: 0, 2580: 0, 7769: 0, 8587: 0, 17312: 0, 13801: None, 15239: 0, 13705: 0, 12804: None, 15406: 0, 2917: 0, 13208: 0, 3969: 0, 11879: None, 1690: 0, 628: 1, 2213: None, 14534: 0, 18567: 0, 289: 0, 10074: 0, 10381: 0, 11363: 0, 17678: 0, 17021: 0, 